In [1]:
import matplotlib
matplotlib.use('Qt5Agg')#for visbrain to work in jupyterlab/notebook
from header import *
import visbrain
from visbrain import Brain, Colorbar, Signal
from visbrain.objects import BrainObj, ColorbarObj, SceneObj, SourceObj
import xarray as xr

In [2]:
task = 'SMEG'
names = ['R_ECG_included', 'R_ECG_excluded', 'T_ECG_included', 'T_ECG_excluded']
name = names[3]
noise_cov = 'baseline_cov'
start = 0.1
stop = 0.2
surface = 'ico4'
sfreq = 200
path = op.join(Analysis_path, task, 'meg', 'Stats', str(start)+'_'+str(stop))
filename = op.join(path, '{}-{}-surface_{}-{}Hz.nc'.format(name, noise_cov, surface, sfreq))
test_key = 'HER_0.0001'
p_thresh = 0.05

In [3]:
stats = xr.open_dataarray(filename, test_key)
stats.load()

<xarray.DataArray (data: 3, hemisphere: 2, time: 30, src: 10242)>
array([[[[ 7.993716, ...,  5.745473],
         ...,
         [10.239217, ...,  6.573741]],

        [[10.370866, ...,  6.368161],
         ...,
         [17.312774, ...,  7.769947]]],


       ...,


       [[[ 1.      , ...,  1.      ],
         ...,
         [ 1.      , ...,  1.      ]],

        [[ 3.      , ...,  3.      ],
         ...,
         [ 3.      , ...,  3.      ]]]])
Coordinates:
  * data        (data) object 'T_stat' 'p_val' 'clu_inds'
  * time        (time) float64 0.05 0.055 0.06 0.065 0.07 0.075 0.08 0.085 ...
  * src         (src) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 ...
  * hemisphere  (hemisphere) object 'left' 'right'

## Order times by number of significant sources

In [6]:
if np.where(stats.loc['p_val']<p_thresh)[1].any():
    t_signif = np.unique(np.where(stats.loc['p_val']<p_thresh)[1], return_counts=True)
    best_t = stats.time[t_signif[0][t_signif[1].argsort()][::-1]]
    print(t_signif)
    print(best_t)
else:
    warnings.warn('\nNo significant cluster.\n')

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]), array([20471, 20460, 20465, 20460, 20455, 20458, 20468, 20457, 20478,
       20479, 20484, 20481, 20482, 20482, 20479, 20480, 20481, 20478,
       20482, 20481, 20484, 20482, 20482, 20482, 20479, 20483, 20483,
       20481, 20481, 20480]))
<xarray.DataArray 'time' (time: 30)>
array([0.15 , 0.1  , 0.18 , 0.175, 0.155, 0.115, 0.14 , 0.11 , 0.16 , 0.165,
       0.105, 0.145, 0.13 , 0.19 , 0.185, 0.125, 0.195, 0.095, 0.12 , 0.17 ,
       0.135, 0.09 , 0.05 , 0.08 , 0.06 , 0.065, 0.055, 0.075, 0.085, 0.07 ])
Coordinates:
  * time     (time) float64 0.15 0.1 0.18 0.175 0.155 0.115 0.14 0.11 0.16 ...


In [ ]:
t = best_t.values[3]
mask = stats.loc['p_val', :, t].values.ravel() < p_thresh
data = stats.loc['T_stat', :, t].values.ravel()[mask]
vertices = np.arange((stats.src.values[-1]+1)*2, dtype=int)[mask]
print('Significant clusters at t =', t, 's:', np.unique(stats.loc['clu_inds', :, t].values.ravel()[mask]))
cmax = np.max(np.abs(data))
clim = (-cmax, cmax)

## Visbrain

In [ ]:
b_obj = BrainObj('inflated', translucent=False, hemisphere='both', sulcus=True)
b_obj.add_activation(data, vertices, hide_under=clim[0], hemisphere='both', smoothing_steps=4, cmap='bwr', clim=clim)
vb = Brain(brain_obj=b_obj, bgcolor=None)

In [ ]:
vb.cbar_select('brain')
vb.cbar_control('brain', cblabel='{} (t = {} s)'.format(test_key, round(t, 3)), bgcolor=None)#, txtcolor='black')
vb.rotate(custom=(225, -45))
vb.show()
#cb.show()
#sc.preview()

In [ ]:
vb.screenshot(op.join(path,'{}_t_{}.png'.format(test_key, t)), transparent=True)

## Cluster analysis

In [ ]:
state = 'RS'
surface = 'ico4'
stc_file = op.join(Analysis_path, task, 'meg', 'SourceEstimate', 'fsaverage', 'Grand_average', '{}-{}-baseline_cov-surface_{}'.format(state, name, surface))
clusters = np.unique(stats.loc['clu_inds'].values[stats.loc['p_val'].values < p_thresh])
clusters

In [ ]:
clu = clusters[0]
vert_clu = np.stack(np.where(stats.loc['clu_inds'].values == clu)[0::2])
vert_clu

In [ ]:
stc = mne.read_source_estimate(stc_file)
data = stc.data.T[:,vert_clu[1]].T
data.shape

In [ ]:
#Signal(data[::10,:-1:6], sf=200, time=stats.time.values, xlabel='Time /s', ylabel='Amplitude /µV', grid_titles=['Source ' + str(k) for k in sorted(vert_clu[1])[::10]]).show()
#Signal(data[::10,:-1:6], form='butterfly', sf=200, time=stats.time.values, xlabel='Time /s', ylabel='Amplitude /µV', grid_titles=['Source ' + str(k) for k in sorted(vert_clu[1])[::10]]).show()
Signal(data.mean(axis=0), time=stc.times*1000, xlabel='Time /ms', ylabel='Amplitude /µV').show()

## Visbrain one hemi at a time

In [4]:
if np.where(stats.loc['p_val', 'left']<p_thresh)[0].any():
    t_signif_lh = np.unique(np.where(stats.loc['p_val', 'left']<p_thresh)[0], return_counts=True)
    best_t_lh = stats.time[t_signif_lh[0][t_signif_lh[1].argsort()][::-1]]
    print(t_signif_lh)
    print(best_t_lh)
else:
    warnings.warn('\nNo significant cluster for left hemisphere.\n')
    best_t_lh = [None]

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]), array([9184, 9402, 9189, 9351, 9257, 9384, 9353, 9569, 9612, 9657, 9683,
       9793, 9832, 9875, 9874, 9897, 9867, 9904, 9923, 9936, 9923, 9885,
       9886, 9921, 9922, 9974, 9953, 9943, 9880, 9834]))
<xarray.DataArray 'time' (time: 30)>
array([0.175, 0.18 , 0.185, 0.145, 0.15 , 0.14 , 0.17 , 0.165, 0.135, 0.125,
       0.16 , 0.155, 0.19 , 0.115, 0.12 , 0.13 , 0.195, 0.11 , 0.105, 0.1  ,
       0.095, 0.09 , 0.085, 0.055, 0.075, 0.08 , 0.065, 0.07 , 0.06 , 0.05 ])
Coordinates:
  * time     (time) float64 0.175 0.18 0.185 0.145 0.15 0.14 0.17 0.165 ...


In [5]:
if np.where(stats.loc['p_val', 'right']<p_thresh)[0].any():
    t_signif_rh = np.unique(np.where(stats.loc['p_val', 'right']<p_thresh)[0], return_counts=True)
    best_t_rh = stats.time[t_signif_rh[0][t_signif_rh[1].argsort()][::-1]]
    print(t_signif_rh)
    print(best_t_rh)
else:
    warnings.warn('\nNo significant cluster for right hemisphere.\n')
    best_t_rh = [None]

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]), array([8727, 8929, 8590, 8772, 8686, 8693, 8837, 9054, 9207, 9249, 9390,
       9499, 9546, 9554, 9552, 9565, 9582, 9590, 9626, 9659, 9638, 9627,
       9592, 9591, 9604, 9675, 9644, 9631, 9583, 9563]))
<xarray.DataArray 'time' (time: 30)>
array([0.175, 0.145, 0.18 , 0.15 , 0.185, 0.155, 0.14 , 0.17 , 0.16 , 0.165,
       0.135, 0.19 , 0.13 , 0.125, 0.195, 0.115, 0.12 , 0.11 , 0.105, 0.1  ,
       0.095, 0.09 , 0.085, 0.055, 0.08 , 0.065, 0.05 , 0.075, 0.07 , 0.06 ])
Coordinates:
  * time     (time) float64 0.175 0.145 0.18 0.15 0.185 0.155 0.14 0.17 0.16 ...


In [6]:
t = float(best_t_rh[0] if best_t_rh[0] is not None else best_t_lh[0])
#t = float(best_t_rh[3])
#t = float(best_t_lh[1])
#t=0.07
print('t =', t, 's')
mask_lh = np.where(stats.loc['p_val', 'left', t] < p_thresh)
mask_rh = np.where(stats.loc['p_val', 'right', t] < p_thresh)
lh = np.where(stats.loc['p_val', 'left', t] < p_thresh, stats.loc['T_stat', 'left', t].values, 0)
rh = np.where(stats.loc['p_val', 'right', t] < p_thresh, stats.loc['T_stat', 'right', t].values, 0)

t = 0.175 s


In [7]:
cmax = np.max(np.abs(np.concatenate([lh, rh])))
clim = (-cmax, cmax)

In [8]:
b_obj = BrainObj('inflated', translucent=False, hemisphere='both', sulcus=True)

#b_obj.add_activation(data=np.where(stats.loc['p_val', 'left', t] < p_thresh, stats.loc['T_stat', 'left', t].values, -np.inf), hide_under=clim[0], vertices=stats.src.values, hemisphere='left', smoothing_steps=4, cmap='bwr', clim=clim)
#b_obj.add_activation(data=np.where(stats.loc['p_val', 'right', t] < p_thresh, stats.loc['T_stat', 'right', t].values, -np.inf), hide_under=clim[0], vertices=stats.src.values, hemisphere='right', smoothing_steps=4, cmap='bwr', clim=clim)
b_obj.add_activation(data=lh, vertices=stats.src.values, hemisphere='left', smoothing_steps=4, cmap='bwr', clim=clim)
b_obj.add_activation(data=rh, vertices=stats.src.values, hemisphere='right', smoothing_steps=4, cmap='bwr', clim=clim)


vb = Brain(brain_obj=b_obj)
#cb = Colorbar(vmin=clim[0], vmax=clim[-1], cmap='bwr', ndigits=4, cblabel='T stat')

INFO | BrainObj(name='inflated') created
INFO | File already dowloaded (/home/benjamin.ador/.local/lib/python3.5/site-packages/visbrain/data/templates/sulcus.npy).
INFO | Add data to secific vertices.
INFO | Add data to secific vertices.
INFO | VolumeObj(name='brodmann') created
INFO | RoiObj(name='brodmann') created
INFO | brodmann ROI loaded.
INFO | CrossSecObj(name='brodmann') created


In [9]:
vb.show()
vb.cbar_select('brain')
vb.cbar_control('brain', cblabel='{} (t = {} s)'.format(test_key, t))
vb.rotate(custom=(225, -45))
#cb.show()
#sc.preview()

In [ ]:
vb.screenshot(op.join(path,'{}_t_{}.png'.format(test_key, t)))

In [ ]:
help(np.concatenate)

In [ ]:
np.stack((stats.src.values, stats.src.values))

In [ ]:
[d for d in data]

In [ ]:
data = stats.loc['T_stat', :, t].values
pval = stats.loc['p_val', :, t].values
np.savetxt('T_stat_2.csv', data, delimiter=',')
np.savetxt('p_val_2.csv', pval, delimiter=',')

In [ ]:
stats.loc['p_val', :, t].values

In [ ]:
(stats.src.values[-1]+1)*2